In [13]:
import requests
import json

url = "https://wahis.woah.org/api/v1/pi/event/filtered-list?language=en"

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept": "application/json, text/plain, */*",
    "Origin": "https://wahis.woah.org",
    "Referer": "https://wahis.woah.org/",
    "authorizationToken": "#PIPRD202006#",
    "clientId": "OIEwebsite",
    "type": "REQUEST"
}

payload = {
    "eventIds": [],
    "reportIds": [],
    "countries": [],
    "firstDiseases": [],
    "secondDiseases": [],
    "typeStatuses": [],
    "animalTypes": [],
    "eventStatuses": [],
    "reasons": [],
    "reportStatuses": [],
    "reportTypes": [],
    "pageNumber": 0,
    "pageSize": 10,
    "sortColumn": "submissionDate",
    "sortOrder": "desc",
    "eventStartDate": None,
    "submissionDate": None
}

response = requests.post(url, headers=headers, json=payload)

print("Status code:", response.status_code)
try:
    print(response.json())
except Exception as e:
    print("Error decoding JSON:", e)
    print(response.text)


Status code: 200
{'list': [{'reportId': 175011, 'createdBy': 'WAHIS', 'eventId': 5825, 'country': 'Greece', 'disease': 'Sheep pox and goat pox ', 'subType': None, 'eventStartDate': '2024-08-19T00:00:00.000+00:00', 'eventStatus': 'On-going', 'reason': 'Recurrence of an eradicated disease', 'reportType': 'FUR', 'reportStatus': 'Validated', 'submissionDate': '2025-06-30T07:00:27.457+00:00', 'reportNumber': 131, 'isAquatic': False}, {'reportId': 175010, 'createdBy': 'WAHIS', 'eventId': 4849, 'country': 'Greece', 'disease': 'African swine fever virus (Inf. with) ', 'subType': None, 'eventStartDate': '2023-01-18T00:00:00.000+00:00', 'eventStatus': 'On-going', 'reason': 'Recurrence of an eradicated disease', 'reportType': 'FUR', 'reportStatus': 'Validated', 'submissionDate': '2025-06-30T06:46:49.284+00:00', 'reportNumber': 85, 'isAquatic': False}, {'reportId': 175005, 'createdBy': 'WAHIS', 'eventId': 5754, 'country': 'Cambodia', 'disease': 'High pathogenicity avian influenza viruses (poultry)

In [20]:
import requests
import csv

url = "https://wahis.woah.org/api/v1/pi/event/filtered-list"  # ⬅️ Removed ?language=en

headers = {
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0",
    "Origin": "https://wahis.woah.org",
    "Referer": "https://wahis.woah.org/",
    "authorizationToken": "#PIPRD202006#",
    "clientId": "OIEwebsite",
    "type": "REQUEST"
}

payload = {
    "countries": [{"label": "Cambodia", "value": "KH"}],
    "pageNumber": 0,
    "pageSize": 10,
    "sortColumn": "submissionDate",
    "sortOrder": "desc"
}

response = requests.post(url, headers=headers, json=payload)

print("Status code:", response.status_code)
try:
    print(response.json())
except Exception as e:
    print("Error parsing JSON:", e)
    print(response.text)


Status code: 400
{'status': 'BAD_REQUEST', 'timestamp': '30-06-2025 10:41:56', 'errorCode': '', 'errorDetail': '', 'exceptionDetail': 'ServerWebInputException'}


In [23]:
!pip install selenium webdriver-manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time

# Set up headless Chrome browser
options = Options()
options.add_argument("--headless")  # comment this line to see the browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Go to the event management page
driver.get("https://wahis.woah.org/#/event-management")

# Wait for the page and data to fully load
time.sleep(15)  # increase if needed

# Get the full rendered HTML
html = driver.page_source

# Save HTML locally to inspect
with open("wahis_event_page.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ Page loaded and saved as 'wahis_event_page.html'")

driver.quit()


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


✅ Page loaded and saved as 'wahis_event_page.html'


In [24]:
from bs4 import BeautifulSoup

with open("wahis_event_page.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

# Example: find all rows containing Cambodia (update the selector as needed)
rows = soup.find_all(string=lambda text: "Cambodia" in text)

for row in rows:
    print("Found:", row)


Found: Cambodia


In [25]:
from bs4 import BeautifulSoup

# Load saved page
with open("wahis_event_page.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

# Try to find all rows that contain 'Cambodia'
# Adjust the tag and class depending on the actual HTML structure
matches = []

# Search all divs or elements that might contain report data
for div in soup.find_all("div"):
    if div.text and "Cambodia" in div.text:
        # Simple keyword match — refine as needed
        matches.append(div.text.strip())

# Output results
for i, m in enumerate(matches, 1):
    print(f"\n--- Report {i} ---\n{m}")



--- Report 1 ---
WAHISmenuAnalyticskeyboard_arrow_downANALYTICS Dashboards enable users to consult, visualize and extract officially validated animal health information.  Disease situation  Quantitative data  Surveillance and control measures  Animal population Reportskeyboard_arrow_downREPORTS Real-time information on exceptional animal disease events for listed and emerging diseases collected via the early warning system, six-monthly information on Listed disease, and annual reports containing animal population information collected via the monitoring system.  Animal disease events  Six-monthly reports  Annual reports WOAHarrow_outwardENFRESAuthorised accessperson_outlinedHOMEkeyboard_double_arrow_rightANIMAL DISEASE EVENTSEvents managementFiltersclosestarApply favoritessaveEvent IDEvent statusAnimal typeReport typeCountryDiseaseSubtypeReason for notificationStart date–Event start dateStart date–Report dateWorld regionfilter_altdouble_arrowFiltersmapSee on the map checkboxCountryexp

In [29]:
from bs4 import BeautifulSoup
import csv

# Load HTML
with open("wahis_event_page.html", "r", encoding="utf-8") as f:
    soup = BeautifulSoup(f, "html.parser")

# Extract all divs or spans containing "Cambodia"
matches = []
for tag in soup.find_all(["div", "span"]):
    text = tag.get_text(strip=True)
    if "Cambodia" in text and "FUR_" in text:
        matches.append(text)

# Check if we found any
print(f"✅ Found {len(matches)} potential Cambodia report(s)")

# Write to CSV
with open("cambodia_reports_from_html.csv", mode="w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([
        "Country", "Report Code", "Disease", "Subtype",
        "Reason", "Event Start", "Submission Date"
    ])

    for line in matches:
        try:
            # Step 1: Start after 'Cambodia'
            parts = line.split("Cambodia")
            rest = parts[1]  # the part after 'Cambodia'

            # Step 2: Extract report code
            report_code = rest.split("High")[0].strip()  # should be FUR_13
            disease_full = "High" + rest.split("High")[1].split("H5N1")[0].strip()
            subtype = "H5N1"
            after_subtype = rest.split("H5N1")[1]

            # Step 3: Find two dates
            import re
            dates = re.findall(r"\d{4}/\d{2}/\d{2}", after_subtype)
            if len(dates) < 2:
                continue
            event_start, submission_date = dates[:2]

            # Step 4: Reason = text between subtype and date
            reason = after_subtype.split(dates[0])[0].strip()

            writer.writerow([
                "Cambodia", report_code, disease_full, subtype,
                reason, event_start, submission_date
            ])
        except Exception as e:
            print("⚠️ Could not parse:", line)
            print("Error:", e)

print("✅ Saved structured Cambodia report(s) to 'cambodia_reports_from_html.csv'")


✅ Found 9 potential Cambodia report(s)
✅ Saved structured Cambodia report(s) to 'cambodia_reports_from_html.csv'
